# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [65]:
# import libraries
import nltk
nltk.download(['stopwords', 'punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import pickle
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
import sklearn.metrics as metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

from sqlalchemy import create_engine

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [34]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con=engine)
#X = 
#Y =
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)

In [36]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [37]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [43]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Tokenize text
    words = text.split()
    
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    return lemmed

    pass

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [44]:
# 2 different pipelines are used 1- random forest and 2-ada boost
pipeline1 =  Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html


In [46]:
pipeline1.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [47]:
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [54]:
def check_results(model, X_test, y_test):
    y_pred = model.predict(X_test)

    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))
        accuracy = (y_pred == y_test.values).mean()
        print('The model accuracy is {:.3f}'.format(accuracy))

In [55]:
check_results(pipeline1, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.63      0.44      0.52      2054
          1       0.84      0.91      0.87      6534
          2       0.33      0.28      0.31        64

avg / total       0.78      0.80      0.78      8652

The model accuracy is 0.945
request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      7180
          1       0.78      0.43      0.55      1472

avg / total       0.87      0.88      0.87      8652

The model accuracy is 0.945
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      8652

The model accuracy is 0.945
aid_related
             precision    recall  f1-score   support

          0       0.76      0.84      0.80      5107
          1       0.73      0.61      0.67      3545

avg / total       0.75 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [56]:
check_results(pipeline2, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.67      0.09      0.16      2054
          1       0.77      0.98      0.86      6534
          2       0.46      0.09      0.16        64

avg / total       0.74      0.77      0.69      8652

The model accuracy is 0.946
request
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      7180
          1       0.76      0.52      0.62      1472

avg / total       0.88      0.89      0.88      8652

The model accuracy is 0.946
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.13      0.05      0.08        38

avg / total       0.99      0.99      0.99      8652

The model accuracy is 0.946
aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      5107
          1       0.75      0.59      0.66      3545

avg / total       0.75 

### 6. Improve your model
Use grid search to find better parameters. 

In [57]:
parameters = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [20, 40]
}

cv1 = GridSearchCV(pipeline1, param_grid=parameters)
cv1

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [58]:
cv2 = GridSearchCV(pipeline2, param_grid=parameters)
cv2

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [61]:
cv1.fit(X_train, y_train)
check_results(cv1, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.68      0.41      0.51      2054
          1       0.83      0.94      0.88      6534
          2       0.31      0.34      0.33        64

avg / total       0.79      0.81      0.79      8652

The model accuracy is 0.947
request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      7180
          1       0.82      0.49      0.62      1472

avg / total       0.89      0.90      0.88      8652

The model accuracy is 0.947
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      8652

The model accuracy is 0.947
aid_related
             precision    recall  f1-score   support

          0       0.78      0.84      0.81      5107
          1       0.75      0.67      0.70      3545

avg / total       0.77 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [62]:
cv2.fit(X_train, y_train)
check_results(cv2, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.66      0.09      0.16      2054
          1       0.77      0.98      0.86      6534
          2       0.47      0.11      0.18        64

avg / total       0.74      0.77      0.69      8652

The model accuracy is 0.946
request
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      7180
          1       0.76      0.50      0.61      1472

avg / total       0.88      0.89      0.88      8652

The model accuracy is 0.946
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.08      0.03      0.04        38

avg / total       0.99      0.99      0.99      8652

The model accuracy is 0.946
aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      5107
          1       0.74      0.59      0.66      3545

avg / total       0.75 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [63]:
# I have already cheched 2 algorithms.

### 9. Export your model as a pickle file

In [66]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv1, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.